# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [35]:
import numpy as np
from scipy import optimize
import sympy as sm
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# The Basic Solow Model

The Basic Solow Model is an exogenous model for a closed economy. The production is determined by two factors in the supply side: physical capital, K, and labor, L. Physical capital is things like machines, factories, buildings and so on, where the labor is the population in the society. Capital and labor accumulate over time. The production can either be consumed today or invested in machines which will increase the production tomorrow. 

**The equations for The Basic Solow Model**
$$Y_t=BK^\alpha_tL^{1-\alpha}_t, \quad 0<\alpha<1 \tag{1}$$
$$K_{t+1}=(1-\delta)K_t+S_t, \quad 0<\delta<1 \tag{2}$$
$$S_t=sY_t, \quad 0<s<1 \tag{3}$$
$$L_{t+1}=(1+n)L_t \tag{4}$$
$$r_t=\alpha B\left(\frac{K_t}{L_t}\right)^{\alpha-1} \tag{5}$$
$$w_t=(1-\alpha)B\left(\frac{K_t}{L_t}\right)^\alpha \tag{6}$$


Where $y_t\equiv\frac{Y_t}{L_t}$ and $k_t\equiv\frac{K_t}{L_t}$.


## Analytical solution

We start by using sympy to find an analytical expression for steady state in The Basic Solow Model. This means that we will solve:
$$k^*=\frac{1}{1+n}(sBk^{*\alpha}+(1-{\delta})k^*)$$
$$y^*=B(k^*)^\alpha

In [25]:
#Define symbols
kstar = sm.symbols('k^*')
ystar = sm.symbols('y^*')
n = sm.symbols('n')
s = sm.symbols('s')
B = sm.symbols('B')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')

In [42]:
#Find steady state for k
ssk = sm.Eq(kstar,((s*B*kstar**alpha+(1-delta)*kstar)/(1+n)))
ss_k = sm.solve(ssk,kstar)[0]
s_s_k = sm.Eq(kstar,ss_k)

#Find steady state for y
ssy = sm.Eq(ystar,((B*ss_k**alpha)))
ss_y = sm.solve(ssy,ystar)[0]
s_s_y = sm.Eq(ystar,ss_y)

#Show the results
display(s_s_k)
display(s_s_y)

Eq(k^*, (B*s/(delta + n))**(-1/(alpha - 1)))

Eq(y^*, B*((B*s/(delta + n))**(-1/(alpha - 1)))**alpha)

In [51]:
#We will now turn our solution into a Python function
ssk_func = sm.lambdify((B, s, delta, n, alpha), ss_k)

ssy_func = sm.lambdify((B, s, delta, n, alpha), ss_y)

#We will now evaluate the functions with specific parameters:
ssk_value = ssk_func(1, 0.2, 0.05, 0.02, 1/3)

ssy_value = ssy_func(1, 0.2, 0.05, 0.02, 1/3)

# Print the values
print('Steady state value of capital: {:.3f}'.format(ssk_value))
print('Steady state value of output: {:.3f}'.format(ssy_value))

Steady state value of capital: 4.829
Steady state value of output: 1.690


## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

We will now find the numerical solution of The Basic Solow Model. 
To do this, we start by defining the parameters. Then we find steady state for k and y using the root finding optimizer, optimize.root_scalar, where we use the method brentq. Brentq is in general considered as the best of the rootfinding methods. This is because that is is a method with a fast convergence speed and it is robust against different types of functions. 

In [60]:
#The parameters
B = 1
s = 0.2
delta = 0.05
n = 0.02
alpha = 1/3

#Finding steady state with an optimizer for k
solow_ss_k = lambda ss_k: ss_k - ((s*B*ss_k**alpha+(1-delta)*ss_k)/(1+n))
result_k = optimize.root_scalar(solow_ss_k,bracket=[0.1,100],method='brentq')

#Finding steady state with an optimizer for y
solow_ss_y = lambda ss_y: ss_y - (B*result_k.root**alpha)
result_y = optimize.root_scalar(solow_ss_y,bracket=[0.1,100],method='brentq')


print('Steady state value of capital: {:.3f}'.format(result_k.root))
print('Steady state value of output: {:.3f}'.format(result_y.root))



Steady state value of capital: 4.829
Steady state value of output: 1.690


As we can see, the numerical solution gives the same steady state values as the analytical solution. 

We will now make a Solow Diagram for The Basic Solow Model.

In [ ]:
# Parametre
B = 1
s = 0.2
delta = 0.05
n = 0.02
alpha = 1/3

# Funktioner
def output(k_t):
    return (n+delta)*k_t

def output(y):
    return B*k_t**alpha

def steady_state():
    return ((s * A) / (n + d)) ** (1 / (1 - alpha))

# Data
k_values = np.linspace(0.1, 10, 100)
y_values = output(k_values)
k_star = effektiv_kapital(k_values)
ss = steady_state()
k_t = [ss] * len(k_values)

# Grafer
fig = make_subplots(rows=2, cols=2, subplot_titles=("Output (Y)", "kt", "k*", "kt over tid"))

fig.add_trace(go.Scatter(x=k_values, y=y_values, mode='lines', name='Output (Y)'), row=1, col=1)
fig.add_trace(go.Scatter(x=k_values, y=k_values, mode='lines', name='kt'), row=1, col=2)
fig.add_trace(go.Scatter(x=k_values, y=k_star, mode='lines', name='k*'), row=2, col=1)
fig.add_trace(go.Scatter(x=k_values, y=k_t, mode='lines', name='kt over tid'), row=2, col=2)

# Layout
fig.update_layout(title="Interaktiv Solow-diagram",
                  xaxis=dict(title="kapital per arbejder (k)"),
                  yaxis=dict(title="Y / kt / k*"),
                  showlegend=False)

# Vis grafer
fig.show()

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

In [38]:
def basic_solow_diagram(k: int, n: float, s: float, B: int, alpha: float, delta: float, kmax: int, kline: float):
    # Calculate the growth rates of capital per capita and the diagonal line
    k_values = np.linspace(0, kmax, 100)
    k_growth = s * B * k_values**alpha
    diagonal = (n + delta) * k_values

    # Calculate steady state capital per capita
    k_star = ((B*s)/(n+delta))**(-1/(alpha-1))

    # The figure size and the two graphs
    plt.figure(figsize=(10, 7))
    plt.plot(k_values, diagonal, label=r'$(n+\delta)k_t$', color='black')
    plt.plot(k_values, k_growth, label=r'$y_t = sBk_t^{\alpha}$', color='mediumorchid')

    #The steady state point, the starting point and the labels:
    plt.axvline(x=kline, linestyle='--', color='royalblue', label=r'$k_0$')
    plt.axvline(x=k_star, linestyle='--', color='gold', label=r'$k*$')
    plt.xlim(0, kmax)
    plt.xlabel('$k_t$')
    plt.ylabel('$k_{t+1}$')
    plt.legend()
    plt.title('The Basic Solow Diagram')
    plt.grid(True)
    
# Interactive plot
interact(basic_solow_diagram,
    k=widgets.fixed(0),
    alpha=widgets.FloatSlider(description=r'α', min=0, max=0.9, step=0.05, value=0.3),
    delta=widgets.FloatSlider(description=r'δ', min=0, max=0.1, step=0.01, value=0.01),
    s=widgets.FloatSlider(description='s', min=0.01, max=0.8, step=0.05, value=0.4),
    n=widgets.FloatSlider(description='n', min=0.01, max=0.1, step=0.005, value=0.01),
    B=widgets.fixed(1),
    kmax=widgets.IntSlider(description='k_t axis', min=1, max=100, step=10, value=100),
    kline=widgets.FloatSlider(description='k_t', min=0, max=100, step=0.1, value=20))

# Show plot
plt.show(basic_solow_diagram)

interactive(children=(FloatSlider(value=0.01, description='n', max=0.1, min=0.01, step=0.005), FloatSlider(val…

# Conclusion

Add concise conclusion. 